# The missing shapes of protected areas

The WDPA reports all protected areas in Europe, but for some there is no information on their geographic shape. In here I want to assess whether those areas can be neglected, and if not, how to estimate them.

In [1]:
import math

import fiona
import geopandas as gpd
import pycountry

## The fraction of protected areas with missing shape

First I am assessing the fraction of protected land, for which the shape is missing. The WDPA comes in two databases: one with polygons reporting the protected areas for which shapes are known, and another one with points of the areas for which shapes are not known. I am not too much interested in the count of areas, but the actual land they cover. Hence I will assess that in the following.

The polygon database is nasty: the polygons are complex and operations often take long. Even worse: there is one polygon which seems invalid and `geopandas` fails reading it. I am hence using `fiona` in the following which allows for quick filtering of the data (the invalid one is not in Europe).

In [2]:
COUNTRIES = [
    "Austria",
    "Belgium",
    "Bulgaria",
    "Croatia",
    "Cyprus",
    "Czech Republic",
    "Denmark",
    "Estonia",
    "Finland",
    "France",
    "Germany",
    "Greece",
    "Hungary",
    "Ireland",
    "Italy",
    "Latvia",
    "Lithuania",
    "Luxembourg",
    "Malta",
    "Netherlands",
    "Poland",
    "Portugal",
    "Romania",
    "Slovakia",
    "Slovenia",
    "Spain",
    "Sweden",
    "United Kingdom",
    "Norway",
    "Switzerland"
]

COUNTRIES_ISO_ALPHA_3 = [pycountry.countries.lookup(country).alpha_3 for country in COUNTRIES]
BBOX_EUROPE = (-12, 20, 40, 79)

In [3]:
with fiona.open("../build/raw-wdpa-jan2018/WDPA_Jan2018-shapefile-points.shp") as f_points:
    rep_area_points = sum([rec[1]["properties"]["REP_AREA"] 
                           for rec in f_points.items(bbox=BBOX_EUROPE)
                           if rec[1]["properties"]["ISO3"] in COUNTRIES_ISO_ALPHA_3
                           if rec[1]["properties"]["STATUS"] == 'Designated'
                          ])
    rep_m_area_points = sum([rec[1]["properties"]["REP_M_AREA"] 
                             for rec in f_points.items(bbox=BBOX_EUROPE)                             
                             if rec[1]["properties"]["ISO3"] in COUNTRIES_ISO_ALPHA_3
                             if rec[1]["properties"]["STATUS"] == 'Designated'
                            ])

In [4]:
print("Reported area of points:        {:.2f} km^2".format(rep_area_points))
print("Reported marine area of points: {:.2f} km^2".format(rep_m_area_points))

Reported area of points:        176613.22 km^2
Reported marine area of points: 241.76 km^2


In [5]:
with fiona.open("../build/raw-wdpa-jan2018/WDPA_Jan2018-shapefile-polygons.shp") as f_polygon:
    rep_area_polygons = sum([rec[1]["properties"]["REP_AREA"] 
                             for rec in f_polygon.items(bbox=BBOX_EUROPE)                           
                             if rec[1]["properties"]["ISO3"] in COUNTRIES_ISO_ALPHA_3
                             if rec[1]["properties"]["STATUS"] == 'Designated'
                            ])
    rep_m_area_polygons = sum([rec[1]["properties"]["REP_M_AREA"] 
                               for rec in f_polygon.items(bbox=BBOX_EUROPE)
                               if rec[1]["properties"]["ISO3"] in COUNTRIES_ISO_ALPHA_3
                               if rec[1]["properties"]["STATUS"] == 'Designated'
                              ])

In [6]:
print("Reported area of polygons:        {:.2f} km^2".format(rep_area_polygons))
print("Reported marine area of polygons: {:.2f} km^2".format(rep_m_area_polygons))

Reported area of polygons:        2376365.87 km^2
Reported marine area of polygons: 876706.27 km^2


In [7]:
fraction_points = rep_area_points / (rep_area_points + rep_area_polygons)
print("Fraction of area reported as points: {:.1f}%".format(fraction_points * 100))

Fraction of area reported as points: 6.9%


In [8]:
fraction_m_points = rep_m_area_points / (rep_m_area_points + rep_m_area_polygons)
print("Fraction of marine area reported as points: {:.1f}%".format(fraction_m_points * 100))

Fraction of marine area reported as points: 0.0%


If I was to ignore the point database, I'd neglect 7% of the protected land in Europe. That's roughly the area of half of the size of Germany. Almost all missing shapes span over land, not marine areas.

## Estimating the shape of areas reported as points

Let's try to come up with an estimation of the shape of areas that are reported as points. Here's what the manual of WDPA Jan 2018 states about the points:

> Where boundary data is unavailable, the latitude and longitude of the centermost point of the site is requested as a reference point for the protected area instead. Although this is strongly encouraged, data providers are not always able to submit such information. Therefore, it should not be assumed that all points in the WDPA represent a central point of a given site. If the protected area is made up of multiple parts, multi- points associated with the central locations of each part of the protected area may be stored instead (see Figure 2.2).

So the points are centroids, or at least should be. A simple approach would be to draw a circle around that centroid with the reported area size and use that as an estimation of the real protected area. A difficulty here would be the areas that are represented by more than one point. Let's find out how many of those there are and how to handle them.

In [9]:
with fiona.open("../build/raw-wdpa-jan2018/WDPA_Jan2018-shapefile-points.shp") as f_points:
    points_in_europe = list(rec for rec in f_points.items(bbox=BBOX_EUROPE)
                            if rec[1]["properties"]["ISO3"] in COUNTRIES_ISO_ALPHA_3
                           )

In [10]:
types = set(point[1]["geometry"]["type"] for point in points_in_europe)
print(types)

{'MultiPoint'}


Technically, everything seems to be stored as a `MultiPoint`.

In [11]:
n_coordinates = set(len(point[1]["geometry"]["coordinates"]) for point in points_in_europe)
print(n_coordinates)

{1}


In any case, there are no entries in the database that contain more than one point.

### Approach 1: Drawing circles around points

In the following, I will project to gall peters temporarily, because the coordinate system is equidistant. Every unit represents 1 meter.

In [12]:
# from http://spatialreference.org/ref/sr-org/22/
GALL_PETERS_PROJ4 = "+proj=cea +lon_0=0 +lat_ts=45 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs "

In [13]:
points = gpd.read_file("../build/raw-wdpa-jan2018/WDPA_Jan2018-shapefile-points.shp")
points_in_europe = points.cx[-12:40, 20:79].loc[points.ISO3.isin(COUNTRIES_ISO_ALPHA_3)].copy()
original_crs = points_in_europe.crs

In [14]:
# convert points to circles
points_in_europe = points_in_europe.to_crs(GALL_PETERS_PROJ4)
def radius_meter(area_squarekilometer):
    area_squaremeter = area_squarekilometer * 1e6
    return math.sqrt(area_squaremeter / math.pi)
points_in_europe.geometry = [rec[1].geometry.buffer(radius_meter(rec[1]["REP_AREA"])) 
                             for rec in points_in_europe.iterrows()]

In [15]:
# test area size (error must be smaller 1%)
area_size_calculated = points_in_europe.area.sum() / 1e6
area_size_reported = points_in_europe.REP_AREA.sum()
assert abs(area_size_calculated - area_size_reported) < area_size_reported / 100

In [16]:
# project back to original CRS
points_in_europe = points_in_europe.to_crs(original_crs)

This approach is simple and pragmatic and its error in terms of total amount of land area is less than 1%.